In [10]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import random



In [19]:
# Start a graph session
sess = tf.Session()


FLAGS = tf.app.flags.FLAGS # 프로젝트 전역으로 변수를 공유합니다
FLAGS.width = 28
FLAGS.height = 28
FLAGS.depth = 1
batch_index = 0
filenames = []


In [20]:
def get_filenames(data_set): # 하위 디렉토리를 돌면서 파일 이름을 가져옵니다
    global filenames
    labels = []

    with open('D:/Eun/CNN4PM/data/ML_Patient monitor data/RCM_PR_1.CSV') as f:
        for line in f: # 파일에서 라인 읽어옴
            inner_list = [elt.strip() for elt in line.split(',')]
            labels += inner_list # label을 리스트에 넣어줍니다
    print(labels[3])

    for i, label in enumerate(labels): # label 디렉토리 안에 파일들을 검색해서 리스트에
        list = os.listdir(FLAGS.data_dir  + '/' + data_set + '/' + label)
        for filename in list:
            # 파일 이름과 label에 해당하는 번호를 묶어서 리스트에 추가합니다
            filenames.append([label + '/' + filename, i]) 

    random.shuffle(filenames) # 파일이름, label 쌍으로 되어있는 리스트를 섞어줍니다

In [13]:
def get_data_jpg(sess, data_set, batch_size):
    global batch_index, filenames

    if len(filenames) == 0: get_filenames(data_set) # 파일 이름들을 읽어오지 않았다면 읽어옵니다
    max = len(filenames)

    # 현재 index 부터 batch_size 만큼 읽어오기 위해 범위를 정해줍니다
    begin = batch_index
    end = batch_index + batch_size

    if end >= max: # 범위가 넘어가면 index를 다시 0으로 설정합니다
        end = max
        batch_index = 0

    x_data = np.array([])
    y_data = np.zeros((batch_size, FLAGS.num_class)) # one hot encoding을 위해 0으로 채워진 리스트를 만듭니다
    index = 0


In [16]:
  for i in range(begin, end):
        # 파일을 얽어옵니다
        with tf.gfile.FastGFile(FLAGS.data_dir + '/' + data_set + '/' + filenames[i][0], 'rb') as f:
            image_data = f.read()

        # JPEG 이미지이므로 decoding을 해야 합니다
        decode_image = tf.image.decode_jpg(image_data, channels=FLAGS.depth)
        resized_image = tf.image.resize_images(decode_image, FLAGS.height, FLAGS.width, method=1) 
        image = sess.run(resized_image)
        
        x_data = np.append(x_data, np.asarray(image.data, dtype='float32')) / 255 
        y_data[index][filenames[i][1]] = 1 
        index += 1

        # 변환된 이미지를 확인하려면 아래 주석을 해제하세요
        print(image.shape, len(image.data))
        im = np.reshape(image.data, (28, 28, 1))
        plt.imshow(im)
        plt.show()

    batch_index += batch_size

    try:
        # 깨진 JPEG 이미지는 여기서 exception이 발생합니다
        # Tensorflow에 넣기 위해 2D로 shape을 변경합니다
        x_data = x_data.reshape(batch_size, FLAGS.height * FLAGS.width * FLAGS.depth)
    except:
        return None, None

    return x_data, y_data

IndentationError: unindent does not match any outer indentation level (<ipython-input-16-779458e33614>, line 21)